In [26]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import yfinance as yf
import pandas_ta as ta
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import MetaTrader5 as mt5
import warnings
warnings.filterwarnings("ignore")
mt5.initialize()
account=51127988
password="Aar2frM7"
server = 'ICMarkets-Demo'

# df = pd.read_csv('HistoricalPrices (1).csv')
def get_rates(pair1, x, tf):
    pair1 = pd.DataFrame(mt5.copy_rates_from_pos(pair1, tf, 0, x))
    pair1['time'] = pd.to_datetime(pair1['time'], unit = 's')
    return pair1[['time', 'open', 'high', 'low', 'close']]

# AUDUSD = get_rates('AUDUSD.a', 2000, mt5.TIMEFRAME_D1)
# AUDUSD = AUDUSD.iloc[:-1]
# df = AUDUSD

In [27]:
# Define a function to calculate the rolling average for a given period
def calculate_rolling_average(data, window):
    return data.rolling(window=window).mean()

In [105]:
df = pd.read_csv('HistoricalPrices (6).csv')
df['Date'] = pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df.tail()

,Date,Open,High,Low,Close
4,2023-09-29,0.6429,0.6502,0.6421,0.6433
3,2023-10-02,0.6433,0.6446,0.6361,0.6362
2,2023-10-03,0.6363,0.6369,0.6285,0.6302
1,2023-10-04,0.6303,0.6343,0.6286,0.6326
0,2023-10-05,0.6324,0.6379,0.6317,0.6372


In [100]:
from sklearn.ensemble import RandomForestRegressor
import xgboost as xgb

In [103]:
from sklearn.metrics import r2_score

def get_prediction_high(df):
    for i in range(1, 7):
        df[f'High_D-{i}'] = df[' High'].shift(i)
    # Calculate the average exchange rate for the last two weeks
    df['Two_Week_Avg'] = calculate_rolling_average(df[' High'], window=14)
    # Calculate the average exchange rate for the last two weeks
    df['Two_Week_Avg'] = calculate_rolling_average(df[' High'], window=14)

    # # Calculate the average exchange rate for the last month
    df['One_Month_Avg'] = calculate_rolling_average(df[' High'], window=30)
    df['RSI'] = ta.rsi(df[' Close'], length=14)
    df.reset_index(drop=True, inplace=True)
    df = df[['Date', ' High', 'High_D-1', 'High_D-2', 'High_D-3', 'High_D-4', 'High_D-5', 'High_D-6', 'Two_Week_Avg', 'One_Month_Avg']]
    df = df.dropna()
    features = ['High_D-1', 'High_D-2', 'High_D-3', 'High_D-4', 'High_D-5', 'High_D-6', 'Two_Week_Avg', 'One_Month_Avg']
    target = ' High'
    X_train, X_test, y_train, y_test = train_test_split(df[features], df[target], test_size=0.2, random_state=42)
    # Create and train a multiple regression model
    model = LinearRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r_squared = r2_score(y_test, y_pred)
    # Predict the exchange rates using the trained model
    df['Predicted_High'] = model.predict(df[features])
    results_df = pd.DataFrame({'Date': df['Date'], 'Actual_Close': df[' High']})
    # Calculate and store predicted values for each day from D to D+14
    for i in range(6):
        day_column = f'High_D-{i+1}'
        results_df[f'Predicted_High_D+{i+1}'] = df[day_column]
        
    # Calculate and store predicted values for each day from D to D+14
    for i in range(6):
        day_column = f'High_D-{i+1}'

    for i in range(6):
        day_column = f'High_D-{i+1}'
        actual_values = df[' High']
        predicted_values = df[day_column]

        r_squared = r2_score(actual_values, predicted_values)
        
    import numpy as np
    import scipy.optimize as opt

    # Define the objective function to minimize (MSE)
    def objective(params):
        predicted = np.dot(X_test, params)
        mse = np.mean((predicted - y_test) ** 2)
        return mse

    # Initialize the parameters for optimization (e.g., as all ones)
    initial_params = np.ones(len(features))

    # Use simulated annealing to optimize the parameters
    result = opt.basinhopping(objective, initial_params, niter=100, stepsize=0.5)

    # Get the optimized parameters
    optimized_params = result.x

    # Re-predict using the optimized parameters
    y_pred_optimized = np.dot(X_test, optimized_params)

    # Calculate the coefficient of determination (R^2) for the optimized predictions
    r_squared_optimized = r2_score(y_test, y_pred_optimized)
    
    print('Unoptimized Prediction:')
    # Get the latest row from your DataFrame
    latest_row = df.iloc[0]

    # Extract the predicted exchange rate for today (Day D)
    exchange_rate_today = latest_row['Predicted_High']

    # Extract the predicted exchange rate for tomorrow (Day D+1)
    predicted_exchange_rate_tomorrow = df['Predicted_High'].iloc[1]

    print(f'rate for today (Day D): {exchange_rate_today}')
    print(f'Predicted exchange rate for tomorrow (Day D+1): {predicted_exchange_rate_tomorrow}')
    # Create and train a Random Forest regression model
    rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
    rf_model.fit(X_train, y_train)

    # Predict using the Random Forest model
    rf_y_pred = rf_model.predict(X_test)

    # Calculate R-squared for Random Forest
    rf_r_squared = r2_score(y_test, rf_y_pred)
    
    df['Predicted_High1'] = rf_model.predict(df[features])
    # Create and train an XGBoost regression model
    xgb_model = xgb.XGBRegressor(n_estimators=100, random_state=42)
    xgb_model.fit(X_train, y_train)

    # Predict using the XGBoost model
    xgb_y_pred = xgb_model.predict(X_test)

    # Calculate R-squared for XGBoost
    xgb_r_squared = r2_score(y_test, xgb_y_pred)
    
    print('Optimized Prediction:')
    exchange_rate_today = df['Predicted_High1'].iloc[0]
    # Extract the predicted exchange rate for tomorrow (Day D+1)
    predicted_exchange_rate_tomorrow = df['Predicted_High1'].iloc[1]

    print(f'rate for today (Day D): {exchange_rate_today}')
    print(f'Predicted exchange rate for tomorrow (Day D+1): {predicted_exchange_rate_tomorrow}')

    

In [104]:
get_prediction_high(df)

Unoptimized Prediction:
rate for today (Day D): 0.7703218496988239
Predicted exchange rate for tomorrow (Day D+1): 0.7711560735537524
Optimized Prediction:
rate for today (Day D): 0.7714980000000007
Predicted exchange rate for tomorrow (Day D+1): 0.770263
